We import what will serve us

In [1]:
import pandas
from pandas import DataFrame, read_csv, to_numeric
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import pickle
from bigml.api import BigML 
import kaggle
from import_export_json import *
from fonction_prep_gmsc import *


%matplotlib notebook

In [2]:
pandas.options.mode.chained_assignment = None

## Loading csv files as data frames

Files must be placed in the same directory as this file. Alternatively, modify the relative path to those files.

In [3]:
#!pwd
#!ls

file_kg_fulltrain = 'cs-training.csv'
file_kg_test =  'cs-test.csv'
dossier_kg_file = './'
dossier_prep_file = './prep_datasets/'
fulltrain=read_csv(dossier_kg_file +file_kg_fulltrain, index_col=0)
test=read_csv(dossier_kg_file +file_kg_test, index_col=0)


In [4]:
project = 'gmsc'
name_files = 'gmsc_files'
version = 'v1.3'

In [5]:
comp_csv_suffix = version + '.csv.bz2'
csv_suffix = version + '.csv'



filename_fulltrain = 'gmsc_fulltrain_' + comp_csv_suffix
filename_train = 'gmsc_train_' + comp_csv_suffix
filename_dev = 'gmsc_dev_' + comp_csv_suffix
filename_test = 'gmsc_test_' + comp_csv_suffix

#### We use RandomState for have a fixed seed and always split the training in the same way

In [6]:
fixed_seed=42
train, dev = train_test_split(fulltrain, test_size=0.2, random_state=fixed_seed)
data_sets=[fulltrain,train,dev,test]
filenames=[filename_fulltrain,filename_train,filename_dev,filename_test]
datasets_names=['fulltrain','train','dev','test']

In [7]:
project_data = json_load_project(project, version)
project_data['prep_data']=[]

In [9]:
project_files = json_load_project(name_files, version)
project_data['files'] = {}
project_data['files']['kaggle_files']= {'train': {'file' :'cs-training.csv', 'dossier' : dossier_kg_file}, 'test': {'file' :'cs-test.csv', 'dossier' : dossier_kg_file}}
project_data['files']['prep_files']= {}

## Correcting the values in monthly income
Set NaN to 0 in the monthly income column

In [10]:
args={'column': 'MonthlyIncome', 'value': 0}
for df in data_sets:
    remplace_fillna(df, **args)
    
project_data['prep_data'].append({'function': 'remplace_fillna', 'arguments': args})

## Creating a new column for monthly debt.

In [11]:
for df in data_sets:
    df.insert(6,"MonthlyDebt",0)
data_sets[0].head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,0,3,0,1,0,1.0
7,0,0.305682,57,0,5710.000000,0.0,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,0,8,0,0,0,0.0
9,0,0.116951,27,0,46.000000,0.0,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,0,9,0,4,0,2.0


We calculate the monthly debt.
If the income is zero, we take the monthly debt from the debt ratio.

In [12]:

for df in data_sets:
    monthly_debt(df)
    
   


project_data['prep_data'].append({'function': 'monthly_debt', 'arguments': args}) 
data_sets[0][['MonthlyDebt','DebtRatio','MonthlyIncome']].head(10)

,MonthlyDebt,DebtRatio,MonthlyIncome
1,7323.197016,0.802982,9120.0
2,316.878123,0.121876,2600.0
3,258.914887,0.085113,3042.0
4,118.963951,0.036050,3300.0
5,1584.975094,0.024926,63588.0
6,1314.624392,0.375607,3500.0
7,5710.000000,NaN,0.0
8,734.790059,0.209940,3500.0
9,46.000000,NaN,0.0
10,14359.393699,0.606291,23684.0


# Create New column disposable income

In [13]:
for df in [fulltrain, test]:
    df.insert(7,"DisposableIncome",0)
fulltrain.head(10)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,0,3,0,1,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,0,8,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,0,9,0,4,0,2.0


In [14]:
for df in data_sets:
    disposable_income(df)
    
project_data['prep_data'].append({'function': 'disposable_income', 'arguments': {}})    
data_sets[0].head(20)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,3,0,1,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,8,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,2,0,0,0,NaN
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,9,0,4,0,2.0


## Correcting the values in number of dependents
Set to 0 the number of dependents when not available.

In [15]:
args = {'column': 'NumberOfDependents', 'value': 0}
for df in data_sets:
    remplace_fillna(df, **args)

project_data['prep_data'].append({'function': 'process_fillna', 'arguments': args})     
data_sets[0][['NumberOfDependents']].head(10)

,NumberOfDependents
1,2.0
2,1.0
3,0.0
4,0.0
5,0.0
6,1.0
7,0.0
8,0.0
9,0.0
10,2.0


# Disposable Income / dept +1


Create new colomn BalancedIncomePerDependent

In [16]:
 for df in data_sets:
    df.insert(8,"BalancedIncomePerDependent",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,3,0,1,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,8,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0,2,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,9,0,4,0,2.0


In [17]:
for df in data_sets:
    balanced_income_per_dependent(df)
    
    
project_data['prep_data'].append({'function': 'balanced_income_per_dependent', 'arguments': {}})   
data_sets[0][['BalancedIncomePerDependent']].head(20)

,BalancedIncomePerDependent
1,598.934328
2,1141.560939
3,2783.085113
4,3181.036049
5,62003.024906
6,1092.687804
7,0.000000
8,2765.209940
9,0.000000
10,3108.202100


# Create new colomn BalancedIncomePerDependent 

Create new colomn  BalancedIncome  MontlyIncome/ (NumberOfDependents +1)

In [18]:
for df in [fulltrain, test]:
    df.insert(8,"BalancedIncome",0)
fulltrain.head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,0,598.934328,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,0,1141.560939,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,0,2783.085113,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,0,3181.036049,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,0,62003.024906,7,0,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,0,1092.687804,3,0,1,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0,0.000000,8,0,3,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,0,2765.209940,8,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0,0.000000,2,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,0,3108.202100,9,0,4,0,2.0


In [19]:
for df in data_sets:
    balanced_income(df)

project_data['prep_data'].append({'function': 'balanced_income', 'arguments': {}})  
data_sets[0]['BalancedIncome'].head(10)

1      3040.000000
2      1300.000000
3      3042.000000
4      3300.000000
5     63588.000000
6      1750.000000
7         0.000000
8      3500.000000
9         0.000000
10     7894.666667
Name: BalancedIncome, dtype: float64

# Weighted sum of late payments

In [20]:
for df in data_sets:
    df.insert(13,"WeightedOfLatePayment",0)
fulltrain


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,WeightedOfLatePayment,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,3040.000000,598.934328,13,0,6,0,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,1300.000000,1141.560939,4,0,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,3042.000000,2783.085113,2,1,0,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,3300.000000,3181.036049,5,0,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,63588.000000,62003.024906,7,0,1,0,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,1750.000000,1092.687804,3,0,1,0,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0.000000,0.000000,8,0,3,0,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,3500.000000,2765.209940,8,0,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0.000000,0.000000,2,0,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,7894.666667,3108.202100,9,0,4,0,0,2.0


In [21]:
for df in data_sets: 
    weighted_of_late_payment(df)
    
project_data['prep_data'].append({'function': 'weighted_of_late_payment', 'arguments': args})
data_sets[0][['WeightedOfLatePayment']].head(10)

,WeightedOfLatePayment
1,2
2,0
3,4
4,0
5,1
6,0
7,0
8,0
9,0
10,0


In [22]:
data_sets[0].head(10)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,MonthlyDebt,DisposableIncome,BalancedIncome,BalancedIncomePerDependent,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,WeightedOfLatePayment,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,7323.197016,1796.802984,3040.000000,598.934328,13,0,6,2,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,316.878123,2283.121877,1300.000000,1141.560939,4,0,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,258.914887,2783.085113,3042.000000,2783.085113,2,1,0,4,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,118.963951,3181.036049,3300.000000,3181.036049,5,0,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,1584.975094,62003.024906,63588.000000,62003.024906,7,0,1,1,0,0.0
6,0,0.213179,74,0,0.375607,3500.0,1314.624392,2185.375608,1750.000000,1092.687804,3,0,1,0,0,1.0
7,0,0.305682,57,0,NaN,0.0,5710.000000,0.000000,0.000000,0.000000,8,0,3,0,0,0.0
8,0,0.754464,39,0,0.209940,3500.0,734.790059,2765.209940,3500.000000,2765.209940,8,0,0,0,0,0.0
9,0,0.116951,27,0,NaN,0.0,46.000000,0.000000,0.000000,0.000000,2,0,0,0,0,0.0
10,0,0.189169,57,0,0.606291,23684.0,14359.393699,9324.606301,7894.666667,3108.202100,9,0,4,0,0,2.0


# Replace the outlier of DebtRatio by 0.3

In [23]:
args = {'debt_value': 0.3}
for df in data_sets:
    debt_ratio_ok(df, **args)
    
    
project_data['prep_data'].append({'function': 'debt_ratio_outlier', 'arguments': args})
data_sets[0]['DebtRatio'].head(10)

1     0.802982
2     0.121876
3     0.085113
4     0.036050
5     0.024926
6     0.375607
7     0.300000
8     0.209940
9     0.300000
10    0.606291
Name: DebtRatio, dtype: float64

# Rearrange columns

We test 6 different configurations

In [27]:
for i in range(4):
    data_sets[i].to_csv(dossier_prep_file+filenames[i],header=True,index_label='Id')
    project_data['files']['prep_files'][datasets_names[i]] = { 'dossier': dossier_prep_file, 'file_name': filenames[i]}
    
project_files

{}

In [28]:
json_save_project(project_data, project, version)
project_data

{'prep_data': [{'function': 'remplace_fillna',
   'arguments': {'column': 'MonthlyIncome', 'value': 0}},
  {'function': 'monthly_debt',
   'arguments': {'column': 'MonthlyIncome', 'value': 0}},
  {'function': 'disposable_income', 'arguments': {}},
  {'function': 'process_fillna',
   'arguments': {'column': 'NumberOfDependents', 'value': 0}},
  {'function': 'balanced_income_per_dependent', 'arguments': {}},
  {'function': 'balanced_income', 'arguments': {}},
  {'function': 'weighted_of_late_payment',
   'arguments': {'column': 'NumberOfDependents', 'value': 0}},
  {'function': 'debt_ratio_outlier', 'arguments': {'debt_value': 0.3}}],
 'files': {'kaggle_files': {'train': {'file': 'cs-training.csv',
    'dossier': './'},
   'test': {'file': 'cs-test.csv', 'dossier': './'}},
  'prep_files': {'fulltrain': {'dossier': './prep_datasets/',
    'file_name': 'gmsc_fulltrain_v1.3.csv.bz2'},
   'train': {'dossier': './prep_datasets/',
    'file_name': 'gmsc_train_v1.3.csv.bz2'},
   'dev': {'dossie

Maud SONET